In [ ]:
import pandas as pd
import numpy as np
!pip install nltk
from nltk.stem import PorterStemmer
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler


: 

In [ ]:
data = pd.read_csv('./content/amazon_alexa.tsv',delimiter='\t',quoting=3)
print(f'Data shape = {data.shape}')

In [ ]:
data.head()

In [ ]:
print(f'Features Name {data.columns.values}')

In [ ]:
data.isnull().sum()

In [ ]:
data[data['verified_reviews'].isna()==True]

In [ ]:
data.dropna(inplace=True)

In [ ]:
print(f'data shape after droping : {data.shape}')

In [ ]:
data['Length'] = data['verified_reviews'].apply(len)

In [ ]:
data.head()

In [ ]:
len(data.iloc[10,3])

In [ ]:
corpus = []
stemmer = PorterStemmer()
for i in range(0,data.shape[0]):
  review = re.sub('[^a-zZ-Z]', ' ',data.iloc[i]['verified_reviews'])
  review = review.lower().split()
  review = [stemmer.stem(word)for word in review if not word in STOPWORDS]
  corpus.append(' '.join(review))

In [ ]:
cv = CountVectorizer(max_features= 2500)
X =  cv.fit_transform(corpus).toarray()
Y = data['feedback'].values

In [ ]:
print(f"X shape : {X.shape}")
print(f"Y shape : {Y.shape}")

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3,random_state = 15)

In [ ]:
print(f"X_train :{X_train.shape}")
print(f"X_test :{X_test.shape}")
print(f"Y_train :{Y_train.shape}")
print(f"Y_test :{Y_test.shape}")

In [ ]:
print(f"X_train max value : {X_train.max()}")
print(f"X_test max value : {X_test.max()}")

In [ ]:

scaler = MinMaxScaler()

In [ ]:
X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.fit_transform(X_test)

In [ ]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train_scl,Y_train)

In [ ]:
print('Training Accuracy :',model_rf.score(X_train_scl,Y_train))
print('Testing Accuracy :',model_rf.score(X_test_scl,Y_test))

In [ ]:
Y_preds = model_rf.predict(X_test_scl)

In [ ]:

cn = confusion_matrix(Y_test,Y_preds)

In [ ]:
cn_display = ConfusionMatrixDisplay(confusion_matrix=cn, display_labels=model_rf.classes_)
cn_display.plot()
plt.show()